In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from speedy_utils.all import *

setup_logger("D")

DEBUG    | utils_print.py:265 (setup_logger) - Logging set to DEBUG


In [33]:





def slow_func(x):
    # try:
    time.sleep(1)  # sleeps for `x` seconds
    logger.info(f"{x=}")
    return x
    # except KeyboardInterrupt:
    #     logger.error("KB Interrupted")
    # except Exception as e:
    #     logger.error(f"Interrupted: {e}")
        

# write a decorator that raises an exception if the function is interrupted by the user


slow_func(1)

INFO     | 3873382941.py:4 (slow_func) - x=1


1

In [82]:
setup_logger("i")

results = None
results = multi_thread(
    func=slow_func,
    orig_inputs=range(20),
    workers=10,
    desc="Demo",
    verbose=False
)

Demo:   0%|                                                      | 0/20 [00:00<?, ?it/s]

INFO     | 3873382941.py:4 (slow_func) - x=1
INFO     | 3873382941.py:4 (slow_func) - x=2
INFO     | 3873382941.py:4 (slow_func) - x=4


Demo:   0%|                                           | 0/20 [00:01<?, ?it/s, SUCCESS=2]

INFO     | 3873382941.py:4 (slow_func) - x=8


Demo:  10%|███▌                               | 2/20 [00:01<00:09,  1.96it/s, SUCCESS=2]

INFO     | 3873382941.py:4 (slow_func) - x=7
INFO     | 3873382941.py:4 (slow_func) - x=5
INFO     | 3873382941.py:4 (slow_func) - x=3
INFO     | 3873382941.py:4 (slow_func) - x=0
INFO     | 3873382941.py:4 (slow_func) - x=9
INFO     | 3873382941.py:4 (slow_func) - x=6


ERROR    | thread.py:302 (multi_thread) - [MAIN PROCESS] Keyboard interrupt detected stop `slow_func` execution.
